In [6]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [23]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,ExtraTreesClassifier
from xgboost import XGBRFClassifier,XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,roc_curve,auc  

In [35]:
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

In [37]:
def drop(dataset):
    data = dataset.drop(columns=['loan_id','financial_institution','origination_date','first_payment_date','number_of_borrowers','m1','m2','m3','m4','m5','m6','m7','m8','m9','m10','m11','m12'])
    
    return data

In [38]:
def eda(dataset):
    data = dataset.copy(deep = True)
    
    for i in ['m1','m2','m3','m4','m5','m6','m7','m8','m9','m10','m11','m12']:
        data[i] = data[i] > 0
        data[i] = data[i].astype('int')
    
    data['last5_sum'] = (data.m9 +data.m10 +data.m11 +data.m12) 
    data['last5'] = (data.m9 +data.m10 +data.m11 +data.m12) > 0
    data['last5'] = data['last5'].astype('int')

    data = drop(data)
    
    data_dummies = pd.get_dummies(data)
    
    return data_dummies

In [46]:
data_train_dummies = eda(data_train)

In [40]:
X_nothing = data_train_dummies.drop('m13',axis= 1)
y = data_train_dummies['m13']

X_train,X_val, y_train,y_val = train_test_split(X_nothing,y, random_state = 0, train_size = 0.9)

In [41]:
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_val = sc.transform(X_val)

In [42]:

model = LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=0.0001,
                           solver='eigen', store_covariance=False, tol=0.0001)

model.fit(X_train, y_train)

y_predict = model.predict(X_val)
# print(confusion_matrix(y_val,y_predict)) 
print(accuracy_score(y_val,y_predict))
    
fpr, tpr, thresholds = roc_curve(y_val, y_predict, pos_label=None)
print('auc :',auc(fpr, tpr))
    
# print(classification_report(y_val, y_predict))

0.9958642081681889
auc : 0.7567029151754305


In [43]:
data_test_dummies = eda(data_test)
data_test_sc = sc.transform(data_test_dummies)

In [44]:
predict_test = model.predict(data_test_sc)

In [57]:
output = pd.DataFrame(data = data_test.loan_id)

In [59]:
output['m13'] = predict_test

In [61]:
output.to_csv('rgd_ldr.csv')